In [1]:
%%capture
!pip install transformers einops

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import requests
import os


In [6]:
model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

configuration_moondream.py:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

moondream.py:   0%|          | 0.00/7.20k [00:00<?, ?B/s]

region_model.py:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
api_key = 'XXXXXXXXXXXXXXXXXX'
endpoint = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'

def translate(text, lang):
    if not text or not lang:
        raise ValueError("Text and target language are required")

    try:
        response = requests.post(
            f"{endpoint}&to={lang}",
            json=[{'Text': text}],
            headers={
                'Ocp-Apim-Subscription-Key': api_key,
                'Ocp-Apim-Subscription-Region': 'southeastasia',
                'Content-Type': 'application/json'
            }
        )

        response.raise_for_status()
        translated_text = response.json()[0]['translations'][0]['text']
        return translated_text

    except requests.exceptions.RequestException as error:
        print('Error translating text:', error)
        return None

In [10]:

image = Image.open('/content/drive/MyDrive/1720934164227801.jpg')
print("Processing your image\n")
enc_image = model.encode_image(image)
lang= input("What language do you want your reply in? Select 'hi' for Hindi, 'en' for english, 'mr' for Marathi and so on.\n").lower()
answer = model.answer_question(enc_image, "Describe the image in as much detail at possible.", tokenizer)

Processing your image

What language do you want your reply in? Select 'hi' for Hindi, 'en' for english, 'mr' for Marathi and so on.
es


In [11]:
if lang=='en':
  print(answer)
else:
  print("Translating...\n")
  trans_answer = translate(answer, lang)
  print(trans_answer)
# answers = model.batch_answer(
#     images=[Image.open('/content/drive/MyDrive/1720934164227801.jpg'), Image.open('/content/drive/MyDrive/HBR.jpg')],
#     prompts=["Describe this image.", "Are there people in this image?"],
#     tokenizer=tokenizer,
# )
#print(answers)

Translating...

La imagen capta un mitin político con un grupo de personas, entre ellas dos hombres y una mujer, de pie frente a una bandera estadounidense. Uno de los hombres levanta el puño en el aire, mientras la mujer sostiene un micrófono. El ambiente parece ser enérgico y apasionado, y los participantes participan activamente en el evento.
